In [1]:
import pandas as pd

- df => icustays
- df2 => admissions
- df3 => patients

In [71]:
df = pd.read_csv('../data/icustays_ibd.csv')

In [6]:
df['subject_id'].unique().shape

(683,)

In [270]:
df2 = pd.read_csv('../data/admissions_ibd.csv')

def parse_race(race):
    if 'WHITE' in race:
        return 'WHITE'
    elif 'BLACK' in race:
        return 'BLACK'
    elif 'HISPANIC' in race or 'LATINO' in race:
        return 'HISPANIC/LATINO'
    elif 'ASIAN' in race:
        return 'ASIAN'
    else:
        return 'OTHER'

df2['race'] = df2['race'].apply(parse_race)

df2['dischtime'] = pd.to_datetime(df2['dischtime'])
df2['deathtime'] = pd.to_datetime(df2['deathtime'])
df2

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10098672,21229395,2142-05-16 04:04:00,2142-05-23 14:15:00,NaT,EW EMER.,P30SZF,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,2142-05-15 15:49:00,2142-05-16 05:45:00,0
1,10098672,21259834,2141-04-13 17:30:00,2141-04-17 19:17:00,NaT,EW EMER.,P77SO2,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,2141-04-13 12:17:00,2141-04-13 18:47:00,0
2,10098672,21921527,2141-06-29 22:55:00,2141-07-03 16:52:00,NaT,URGENT,P816G0,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,NaN,NaN,0
3,10098672,24843492,2142-05-06 07:15:00,2142-05-11 13:39:00,NaT,SURGICAL SAME DAY ADMISSION,P857XC,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,NaN,NaN,0
4,10098672,26570321,2142-01-20 23:24:00,2142-01-28 18:47:00,NaT,EW EMER.,P95PIA,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,2142-01-20 19:54:00,2142-01-21 01:07:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10605,16308779,27539941,2138-01-02 04:15:00,2138-01-04 16:35:00,NaT,EU OBSERVATION,P886Y0,EMERGENCY ROOM,NaN,Medicare,ENGLISH,WIDOWED,WHITE,2138-01-01 20:23:00,2138-01-02 04:51:00,0
10606,10556676,20224885,2167-04-16 15:33:00,2167-04-17 09:30:00,NaT,EU OBSERVATION,P76VMW,EMERGENCY ROOM,NaN,Other,ENGLISH,MARRIED,WHITE,2167-04-16 07:16:00,2167-04-17 09:30:00,0
10607,10556676,22783191,2164-02-12 04:35:00,2164-02-14 12:45:00,NaT,EW EMER.,P534S8,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2164-02-11 17:41:00,2164-02-12 05:11:00,0
10608,10556676,25577156,2163-10-31 21:05:00,2163-12-19 18:00:00,NaT,EW EMER.,P534S8,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Other,ENGLISH,MARRIED,WHITE,2163-10-31 18:28:00,2163-10-31 23:48:00,0


In [271]:
hadm_ids_dead = df2[df2['deathtime'] == df2['dischtime']]['hadm_id']

In [272]:
hadm_ids_dead.shape

(103,)

In [273]:
# 死亡记录
df_dead = df[df['hadm_id'].isin(hadm_ids_dead)]

In [274]:
df_alive = df[~df['hadm_id'].isin(hadm_ids_dead)]

In [275]:
df_dead['hadm_id'].unique().shape

(84,)

In [276]:
df_alive['hadm_id'].unique().shape

(1077,)

In [277]:
# admissions 中有死亡记录 
# icustats 中有死亡记录
# 推断: 死在 ICU
df2[df2['deathtime'] == df2['dischtime']][df2[df2['deathtime'] == df2['dischtime']]['hadm_id'].isin(df_dead['hadm_id'])].shape

(84, 16)

In [278]:
# admissions 中有死亡记录 
# icustats 没中有死亡记录
# 推断: 不是死在 ICU
df2[df2['deathtime'] == df2['dischtime']][~df2[df2['deathtime'] == df2['dischtime']]['hadm_id'].isin(df_dead['hadm_id'])].shape

(19, 16)

In [279]:
df3 = pd.read_csv('../data/patients_ibd.csv')
df3['anchor_year'] = df3['anchor_year'].apply(lambda x: pd.to_datetime('{}-01-01'.format(x)))
df3.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10098672,M,61,2140-01-01,2011 - 2013,NaN
1,10303503,F,23,2144-01-01,2008 - 2010,NaN
2,10312715,M,39,2176-01-01,2008 - 2010,NaN
3,10318500,F,46,2194-01-01,2011 - 2013,NaN
4,10410021,M,49,2135-01-01,2011 - 2013,NaN


In [280]:
df2_dead = df2[df2['deathtime'] == df2['dischtime']]
df2_alive = df2[~(df2['deathtime'] == df2['dischtime'])]

In [281]:
tmp = df2_dead.merge(df3, on='subject_id')
tmp['d_age'] = (tmp['deathtime'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']

tmp['d_age'].mean()

70.96965021944409

> admissions 和 icustays 两张表格

admissions 中一共有 103 个病人有死亡记录

- 其中 有 84 个病人是在 icustays 中有记录，暂且认为是死在了 ICU

- 另外 有 19 个病人不在 icustays 中有记录，暂且认为是死在了其他地方

In [282]:
# 84 个死在 ICU 的平均死亡年龄
tmp = df2_dead[df2_dead['subject_id'].isin(df_dead['subject_id'])].merge(df3, on='subject_id')
tmp['d_age'] = (tmp['deathtime'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']

tmp['d_age'].mean(), tmp['d_age'].std()

(70.99500978473581, 13.239634625711627)

In [283]:
# 19 个不是死在 ICU 的平均死亡年龄
tmp = df2_dead[~df2_dead['subject_id'].isin(df_dead['subject_id'])].merge(df3, on='subject_id')
tmp['d_age'] = (tmp['deathtime'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']

tmp['d_age'].mean(), tmp['d_age'].std()

(70.85753424657534, 16.31911577724258)

**惊奇的发现，死在了 ICU 的病人和 不是死在了 ICU 的病人的平均死亡年龄非常接近！**

分别是 70.99 岁和 70.85 岁

> Range: 683 个来过 ICU 的病人

| | 没死 | 死了 | 死在 ICU | 死在别的地方 |
|-| -- | --- | -------- | ------------- |
|人数| 588 | 95 |  84 | 11 |
| AVG LOS | 5.94 | 8.47   | 9.24 | 5.88 |

In [284]:
stats = df.groupby('subject_id').agg({
    'los': ['count', 'sum']
})

stats.columns = ['count', 'sum']
stats = stats.reset_index()

In [285]:
stats['dead'] = stats['subject_id'].isin(df2_dead['subject_id'])
stats['dead_icu'] = stats['subject_id'].isin(df_dead['subject_id'])

In [286]:
stats[~stats['dead']]['sum'].mean()

5.947254716836735

In [287]:
stats[stats['dead']]['sum'].mean()

8.47324610526316

In [288]:
stats[stats['dead_icu']]['sum'].mean()

9.240651583333337

In [289]:
stats[~stats['dead_icu']]['sum'].mean()

5.886025743739566

In [293]:
for col in ['language', 'marital_status', 'race', 'insurance']:
    print('- total')
    print(df2[col].value_counts())
    print(df2[col].value_counts() / df2[col].value_counts().sum())
    print('- dead')
    print(df2_dead[col].value_counts())
    print(df2_dead[col].value_counts() / df2_dead[col].value_counts().sum())
    print()

- total
language
ENGLISH    10252
?            358
Name: count, dtype: int64
language
ENGLISH    0.966258
?          0.033742
Name: count, dtype: float64
- dead
language
ENGLISH    94
?           9
Name: count, dtype: int64
language
ENGLISH    0.912621
?          0.087379
Name: count, dtype: float64

- total
marital_status
MARRIED     4773
SINGLE      4167
WIDOWED      831
DIVORCED     802
Name: count, dtype: int64
marital_status
MARRIED     0.451433
SINGLE      0.394117
WIDOWED     0.078596
DIVORCED    0.075854
Name: count, dtype: float64
- dead
marital_status
MARRIED     55
SINGLE      21
WIDOWED     13
DIVORCED     9
Name: count, dtype: int64
marital_status
MARRIED     0.561224
SINGLE      0.214286
WIDOWED     0.132653
DIVORCED    0.091837
Name: count, dtype: float64

- total
race
WHITE              8776
BLACK               931
OTHER               495
HISPANIC/LATINO     329
ASIAN                79
Name: count, dtype: int64
race
WHITE              0.827144
BLACK              0.08774

In [294]:
df3

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10098672,M,61,2140-01-01,2011 - 2013,NaN
1,10303503,F,23,2144-01-01,2008 - 2010,NaN
2,10312715,M,39,2176-01-01,2008 - 2010,NaN
3,10318500,F,46,2194-01-01,2011 - 2013,NaN
4,10410021,M,49,2135-01-01,2011 - 2013,NaN
...,...,...,...,...,...,...
2412,19878969,F,85,2149-01-01,2008 - 2010,2151-04-25
2413,13561252,M,87,2148-01-01,2008 - 2010,2151-10-05
2414,16308779,F,91,2138-01-01,2008 - 2010,2140-12-11
2415,10556676,F,53,2163-01-01,2011 - 2013,NaN
